In [1]:
# LOADING MODULES

import numpy as np
import pandas as pd

C:\Users\hmitt\AppData\Local\Temp\ipykernel_40032\447460998.py:18: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr
C:\Users\hmitt\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Import data

In [2]:
train = pd.read_csv('C:\\Users\\hmitt\\Downloads\\new-york-city-taxi-fare-prediction (1)\\train.csv', nrows = 2000000)
train.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [3]:
train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,2.000000e+06,2.000000e+06,2.000000e+06,1.999986e+06,1.999986e+06,2.000000e+06
mean,1.134779e+01,-7.252321e+01,3.992963e+01,-7.252395e+01,3.992808e+01,1.684113e+00
std,9.852883e+00,1.286804e+01,7.983352e+00,1.277497e+01,1.032382e+01,1.314982e+00
min,-6.200000e+01,-3.377681e+03,-3.458665e+03,-3.383297e+03,-3.461541e+03,0.000000e+00
25%,6.000000e+00,-7.399208e+01,4.073491e+01,-7.399141e+01,4.073400e+01,1.000000e+00
50%,8.500000e+00,-7.398181e+01,4.075263e+01,-7.398016e+01,4.075312e+01,1.000000e+00
75%,1.250000e+01,-7.396713e+01,4.076710e+01,-7.396369e+01,4.076809e+01,2.000000e+00
max,1.273310e+03,2.856442e+03,2.621628e+03,3.414307e+03,3.345917e+03,2.080000e+02


# Data Preprocessing

Performing basic data cleaning which involves:

* Checking for missing values and removing the rows corresponding to them if any.
* Checking for duplicate values and removing the rows corresponding to them if any.
* Removing the rows that have -ve values of Fare amount as Fare cannot be -ve.
* Truncating longitude and latitude values to fit NY coordinates.

In [4]:
train.isnull().sum()

key                   0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude    14
dropoff_latitude     14
passenger_count       0
dtype: int64

In [6]:
train=train[train['dropoff_latitude'].notnull()]

In [7]:
train

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02 UTC,-73.950787,40.770836,-73.951043,40.774543,1
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28 UTC,-73.983035,40.748276,-73.974725,40.741978,1
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41 UTC,-74.003482,40.743577,-73.976758,40.765265,1
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37 UTC,-73.971819,40.797243,-73.985921,40.752459,3


In [8]:
train.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [9]:
train.duplicated().sum()

0

In [10]:
train=train[train['fare_amount']>=0]

In [11]:
train

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02 UTC,-73.950787,40.770836,-73.951043,40.774543,1
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28 UTC,-73.983035,40.748276,-73.974725,40.741978,1
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41 UTC,-74.003482,40.743577,-73.976758,40.765265,1
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37 UTC,-73.971819,40.797243,-73.985921,40.752459,3


In [12]:
#reading the train data and seeing what the boundaries are.
print("Longitude Boundary in train data")
print (min(train.pickup_longitude.min(), train.dropoff_longitude.min()),max(train.pickup_longitude.max(), train.dropoff_longitude.max()))

print("Latitude Boundary in train data")
print (min(train.pickup_latitude.min(), train.pickup_latitude.min()),max(train.pickup_latitude.max(), train.pickup_latitude.max()))


Longitude Boundary in train data
-3383.296608 3414.306675
Latitude Boundary in train data
-3458.664702 2621.62843


Used the link to obtain the Boundary Co-ordinates of New York https://www.mapdevelopers.com/geocode_bounding_box.php

The boundaries of New York are:

North Latitude: 40.917577

South Latitude: 40.477399

East Longitude: -73.700272

West Longitude: -74.259090

Useing the above values to build a boundary for the allowed Drop_off and Pick_up Longitude, Latittudes.

In [13]:
boundary={'north_lat':40.917577,
          'south_lat':40.477399,
          'east_long':-73.700272, 
          'west_long':-74.259090}

In [ ]:
train.loc[~((train.pickup_longitude >= boundary['west_long'] ) & (train.pickup_longitude <= boundary['east_long']) &
            (train.pickup_latitude >= boundary['south_lat']) & (train.pickup_latitude <= boundary['north_lat']) &
            (train.dropoff_longitude >= boundary['west_long']) & (train.dropoff_longitude <= boundary['east_long']) &
            (train.dropoff_latitude >=boundary['south_lat']) & (train.dropoff_latitude <= boundary['north_lat'])),'is_outlier_loc']=1
train.loc[((train.pickup_longitude >= boundary['west_long'] ) & (train.pickup_longitude <= boundary['east_long']) &
            (train.pickup_latitude >= boundary['south_lat']) & (train.pickup_latitude <= boundary['north_lat']) &
            (train.dropoff_longitude >= boundary['west_long']) & (train.dropoff_longitude <= boundary['east_long']) &
            (train.dropoff_latitude >=boundary['south_lat']) & (train.dropoff_latitude <= boundary['north_lat'])),'is_outlier_loc']=0

# print("Outlier vs Non Outlier Counts")
# print(train['is_outlier_loc'].value_counts())

train=train.loc[train['is_outlier_loc']==0]
train.drop(['is_outlier_loc'],axis=1,inplace=True)
print (train.shape)

In [14]:
train = train.loc[(train.pickup_latitude >= boundary['south_lat']) & (train.pickup_latitude <= boundary['north_lat'])]
train = train.loc[(train.dropoff_latitude >= boundary['south_lat']) & (train.dropoff_latitude <= boundary['north_lat'])]
train = train.loc[(train.pickup_longitude >= boundary['west_long']) & (train.pickup_longitude <= boundary['east_long'])]
train = train.loc[(train.dropoff_longitude >= boundary['west_long']) & (train.dropoff_longitude <= boundary['east_long'])]

In [15]:
train

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02 UTC,-73.950787,40.770836,-73.951043,40.774543,1
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28 UTC,-73.983035,40.748276,-73.974725,40.741978,1
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41 UTC,-74.003482,40.743577,-73.976758,40.765265,1
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37 UTC,-73.971819,40.797243,-73.985921,40.752459,3


In [16]:
train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1.955584e+06,1.955584e+06,1.955584e+06,1.955584e+06,1.955584e+06,1.955584e+06
mean,1.128871e+01,-7.397554e+01,4.075082e+01,-7.397463e+01,4.075118e+01,1.684452e+00
std,9.480855e+00,3.440604e-02,2.688256e-02,3.389022e-02,3.075060e-02,1.306931e+00
min,0.000000e+00,-7.425882e+01,4.048254e+01,-7.425858e+01,4.047791e+01,0.000000e+00
25%,6.000000e+00,-7.399229e+01,4.073654e+01,-7.399160e+01,4.073552e+01,1.000000e+00
50%,8.500000e+00,-7.398211e+01,4.075333e+01,-7.398063e+01,4.075382e+01,1.000000e+00
75%,1.250000e+01,-7.396839e+01,4.076748e+01,-7.396543e+01,4.076837e+01,2.000000e+00
max,5.000000e+02,-7.370044e+01,4.091748e+01,-7.370039e+01,4.091754e+01,9.000000e+00


In [17]:
train.to_csv("train_dp.csv")